### <b> 단기 투자 시 수익 성과가 좋은 팩터 전략 <b>
    1. 밸류 팩터(1m) : 저pbr, 저per (단, per>0이고 pbr>0인 종목)
    2. 어닝 모멘텀(1m) : EPS 변화율이 높은 종목
    3. 가격 모멘텀
    - 이격도 리버전 : 단기 투자 전략에 주가/직전 1개월 평균주가 하위 종목을 선택합니다 
    - 주가 리버전 (1m) : 직전 1개월 주가 수익률 하위 종목 투자
    4. 트레이딩 팩터 : 개인 순매수 (1m)

In [8]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)
import json
import requests

from pykrx import stock

### 1.가치주 : 저per, 저pbr

In [9]:
data = stock.get_market_cap("20240119", market="ALL") #3개월의 투자 수익률 확인 위해
data = data[['종가', '시가총액']]
data.head()

,종가,시가총액
티커,,
005930,74700,445942756485000
000660,141300,102866734174500
373220,383500,89739000000000
207940,770000,54803980000000
005935,60300,49620068010000


In [10]:
#api 활용 데이터 - per, pbr
df = stock.get_market_fundamental_by_ticker("20240119", market='ALL')
df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
060310,947,117.50,3.72,30,0.00,0
095570,8076,23.06,0.57,201,5.83,270
006840,41948,0.00,0.39,0,1.22,200
054620,10864,16.10,0.75,505,0.00,0
265520,17980,4.10,1.25,5463,1.21,270


In [11]:
df.describe() #per<5 고정 #PBR의 경우 상위 25% 값보다 작은 것

,BPS,PER,PBR,EPS,DIV,DPS
count,2.606000e+03,2606.000000,2606.000000,2606.000000,2606.000000,2606.000000
mean,2.209889e+04,29.720652,2.357943,1726.657713,1.320388,344.605142
std,1.208338e+05,222.339143,15.335183,10168.974801,2.190288,1162.628930
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.456250e+03,0.000000,0.480000,0.000000,0.000000,0.000000
50%,4.807000e+03,5.435000,0.975000,143.500000,0.000000,0.000000
75%,1.265075e+04,16.970000,1.990000,939.000000,1.967500,200.000000
max,4.906107e+06,6890.000000,741.180000,406090.000000,34.200000,20000.000000


In [12]:
#업데이트 편의를 위해 넘파이 사용
import numpy as np
column_pbr = df['PBR'].values
pbr_25 = np.percentile(column_pbr, 25)

df2 = df.loc[(df['PER'] < 5) & (df['PBR'] < pbr_25)] #저per, 저pbr인 데이터
df2 = df2[['PER', 'PBR']]
df2

,PER,PBR
티커,,
006840,0.00,0.39
138930,2.91,0.23
001465,0.00,0.00
035760,0.00,0.36
00104K,0.00,0.00
...,...,...
028080,0.00,0.47
079980,0.00,0.35
005010,0.84,0.29


In [13]:
df2.index

Index(['006840', '138930', '001465', '035760', '00104K', '011155', '001045',
       '097955', '000480', '456440',
       ...
       '008775', '002460', '133820', '000850', '032560', '028080', '079980',
       '005010', '000540', '000545'],
      dtype='object', name='티커', length=488)

In [14]:
df3 = pd.merge(data, df2, on='티커')
df3

,종가,시가총액,PER,PBR
티커,,,,
005935,60300,49620068010000,0.00,0.00
105560,49500,19973798064000,4.52,0.40
055550,36800,18869548532800,4.35,0.39
011200,19220,13243339113120,0.94,0.45
086790,41200,12045091837600,3.49,0.33
...,...,...,...,...
001525,5940,3669322140,0.00,0.00
002787,12410,3658567280,0.00,0.00
002785,4230,3626032140,0.00,0.00


### 1.2 PER > 0, PBR > 0

In [15]:
df3 = df3.loc[(df['PER'] > 0) & (df['PBR'] > 0)]
df3 = df3[['종가', 'PER', 'PBR']]
df3.shape 

(153, 3)

In [16]:
df3.index

Index(['105560', '055550', '011200', '086790', '316140', '024110', '004020',
       '078930', '011780', '138930',
       ...
       '006200', '127710', '050120', '012620', '000950', '225590', '054940',
       '106240', '377460', '115570'],
      dtype='object', name='티커', length=153)

   ### 3. EPS 변화율 
   - 1m 기준

In [22]:
#eps변화율 분석(1m)
df = stock.get_market_fundamental("20231203", "20240104", '006840', freq="m")
df = df.T
df

날짜,2023-12-31,2024-01-31
BPS,41948.00,41948.00
PER,0.00,0.00
PBR,0.42,0.41
EPS,0.00,0.00
DIV,1.15,1.17
DPS,200.00,200.00


In [23]:
import pandas as pd

ticker = df3.index
eps_change_data = []

for stock_code in ticker:
    df = stock.get_market_fundamental("20231203", "20240104", stock_code, freq="m")
    df = df.T
    df['변화율'] = (df.iloc[:, 1] - df.iloc[:, 0]) / df.iloc[:, 0]
    eps_change_data.append(df['변화율'])

# DataFrame 생성
eps_change_df = pd.DataFrame(eps_change_data, index=ticker)
eps_change_df

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
105560,0.0,0.040426,0.048781,0.0,-0.040140,0.0
055550,0.0,0.068966,0.051282,0.0,-0.065836,0.0
011200,0.0,0.219512,0.225000,0.0,-0.184615,0.0
086790,0.0,0.028329,0.030303,0.0,-0.024907,0.0
316140,0.0,-0.003257,0.000000,0.0,0.002278,0.0
...,...,...,...,...,...,...
225590,0.0,-0.029762,-0.040816,0.0,NaN,NaN
054940,0.0,-0.002558,0.000000,0.0,NaN,NaN
106240,0.0,0.083333,0.064516,0.0,NaN,NaN


In [24]:
eps_change_df['EPS'].mean()

0.0

In [25]:
eps_change_df = eps_change_df.loc[eps_change_df['EPS'] <= 0]
ticker = eps_change_df.index.to_list()
ticker

['105560',
 '055550',
 '011200',
 '086790',
 '316140',
 '024110',
 '004020',
 '078930',
 '011780',
 '138930',
 '088350',
 '036460',
 '175330',
 '028670',
 '139480',
 '000880',
 '047040',
 '004170',
 '375500',
 '139130',
 '137310',
 '006360',
 '001120',
 '009970',
 '000670',
 '003380',
 '023590',
 '016380',
 '006120',
 '003240',
 '001720',
 '007700',
 '383800',
 '032190',
 '020000',
 '001940',
 '058650',
 '377740',
 '017940',
 '123890',
 '151860',
 '002310',
 '002030',
 '093050',
 '005810',
 '306200',
 '007690',
 '003300',
 '035890',
 '003960',
 '084010',
 '036530',
 '003800',
 '029460',
 '003120',
 '000480',
 '104700',
 '013120',
 '005010',
 '001500',
 '009160',
 '030530',
 '213500',
 '001230',
 '037710',
 '018310',
 '002810',
 '001270',
 '016590',
 '035510',
 '004360',
 '030210',
 '002020',
 '007160',
 '363280',
 '038540',
 '003200',
 '035080',
 '009580',
 '003070',
 '000540',
 '023760',
 '317400',
 '007860',
 '014280',
 '104480',
 '092790',
 '039560',
 '023600',
 '000700',
 '121440',

### 4. 주가 리버전 (1m)
- 직전 1 달간 주가수익률 하위 투자 . 주가 모멘텀 (1m) 팩터의 반대

In [34]:
df = stock.get_market_ohlcv("20231219", "20240119", '024110', "m") #직전 한달 주가 수익률
gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100
gain

-1.93929173693086

In [35]:
df

,시가,고가,저가,종가,거래량
날짜,,,,,
2023-12-31,11800,11910,11560,11860,10702107
2024-01-31,11740,11820,11350,11630,10979931


In [36]:
import pandas as pd

tickers = eps_change_df.index.to_list()
gains = []

for ticker in tickers:
    df = stock.get_market_ohlcv("20231219", "20240119", ticker, "m")
    gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100
    gains.append(gain)

result_df = pd.DataFrame({'ticker': tickers, 'gain': gains})
result_df.set_index('ticker', inplace=True)

In [37]:
result_df.describe()

,gain
count,153.000000
mean,-3.619749
std,5.289040
min,-16.403311
25%,-6.122449
50%,-3.916084
75%,-1.424051
max,14.065934


In [38]:
gain_25 = result_df['gain'].quantile(0.25)
selected_rows = result_df['gain'] < gain_25

temp_df = result_df[selected_rows]
temp_df.index.nunique() #36개의 종목 필터링

38

In [39]:
temp_df.index

Index(['105560', '055550', '004020', '011780', '088350', '036460', '175330',
       '139480', '000880', '004170', '137310', '001120', '009970', '000670',
       '016380', '007700', '306200', '003120', '000480', '001230', '004360',
       '030210', '003070', '317400', '007860', '092790', '012700', '067990',
       '053700', '005960', '013580', '004890', '007980', '101330', '091590',
       '140520', '000950', '106240'],
      dtype='object', name='ticker')

### 5. 이격도 리버전 (1m)
- 저이격도 선택

In [40]:
data = stock.get_market_ohlcv("20231219", "20240119", "011200", "d")
mean = data['종가'].mean()
close_temp = stock.get_market_ohlcv("20240119","20240119", "011200", "d")
close = close_temp.iloc[0,3]
revision = (close / mean) * 100
revision

95.99714591509097

In [41]:
tickers = temp_df.index.tolist()
revisions = []

for ticker in tickers:
    data = stock.get_market_ohlcv("20231219", "20240119", ticker, "d")
    mean = data['종가'].mean()
    close_temp = stock.get_market_ohlcv("20240119","20240119", ticker, "d")
    close = close_temp.iloc[0,3]
    revision = (close / mean) * 100
    revisions.append(revision)

# 새로운 데이터프레임 생성
result_df = pd.DataFrame({'ticker': tickers, 'revision': revisions})
result_df.set_index('ticker', inplace=True)

In [42]:
result_df.describe()

,revision
count,38.000000
mean,93.479831
std,2.348191
min,87.549251
25%,91.897306
50%,93.724039
75%,94.834295
max,98.247782


In [43]:
revision_50 = result_df['revision'].quantile(0.5)
selected_rows = result_df['revision'] < revision_50

revision_df = result_df[selected_rows]
revision_df.index.nunique() 

19

In [44]:
revision_df.index

Index(['004020', '011780', '088350', '004170', '137310', '000670', '016380',
       '007700', '003120', '000480', '001230', '030210', '003070', '092790',
       '005960', '004890', '007980', '091590', '000950'],
      dtype='object', name='ticker')

### 6. 개인 순매수 (1m)
- 직전 한달 간 개인 순매수대금 / 시가총액 *100
- 고개인매수 종목

In [45]:
data1 = stock.get_market_trading_value_by_date("20231219", "20240119", '024110')[['기관합계','개인']] #직전 한달간 개인 순매수 대금
data2 = stock.get_market_cap("20231219", "20240119", '024110')['시가총액']
data1['개인순매수'] = (data1['개인'] / data2) * 100
data1 = data1[['개인', '개인순매수']]
data1['개인순매수'].mean()

0.0054153607562867235

In [46]:
tickers = ['011200', '000670', '003240', '005880', '000540', '013120', '044450',
       '363280', '317400', '104480', '053700', '091590', '004870', '050860',
       '290120', '050120', '073540', '115570']
mean_values = []

for ticker in tickers:
    data1 = stock.get_market_trading_value_by_date("20231219", "20240119", ticker)
    data2 = stock.get_market_cap("20231219", "20240119", ticker)['시가총액']
    data1['개인순매수'] = (data1['개인'] / data2) * 100
    mean = data1['개인순매수'].mean()
    mean_values.append(mean)

final_df = pd.DataFrame({'ticker': tickers, 'mean': mean_values})
final_df.set_index('ticker', inplace=True)
final_df

,mean
ticker,
011200,-0.047920
000670,0.003378
003240,0.004105
005880,0.086930
000540,0.002260
013120,0.002326
044450,-0.032182
363280,0.062050
317400,0.023633


In [50]:
final_df.describe()

,mean
count,18.000000
mean,0.007256
std,0.030461
min,-0.047920
25%,-0.003443
50%,0.002293
75%,0.013497
max,0.086930


In [55]:
final_50 = final_df['mean'].quantile(0.5)
selected_rows = final_df['mean'] < final_50
screening = final_df[selected_rows]
screening.index

Index(['011200', '000540', '044450', '053700', '091590', '004870', '290120',
       '050120', '073540'],
      dtype='object', name='ticker')

### 백테스트

In [242]:
#백테스트를 위한 코드
from pykrx import stock
import FinanceDataReader as fdr

ref = fdr.DataReader(
    symbol = 'KS11',
    start = '20240119',
    end = '20240320') #3개월 간 코스피 지수 수익률

ref['agr'] = (ref['Close'] / ref['Open'])
ref

,Open,High,Low,Close,Volume,Change,UpDown,Comp,Amount,MarCap,agr
Date,,,,,,,,,,,
2024-01-19,2468.43,2479.00,2455.50,2472.74,518806918,0.0134,1,32.70,9174537029762,2005853267137231,1.001746
2024-01-22,2489.57,2490.69,2464.35,2464.35,388045571,-0.0034,2,-8.39,8419915774574,1999139797919650,0.989870
2024-01-23,2478.32,2482.84,2464.24,2478.61,472284165,0.0058,1,14.26,8029400288499,2010744285970351,1.000117
2024-01-24,2476.22,2476.22,2454.34,2469.69,395463694,-0.0036,2,-8.92,7446526589060,2003932327419686,0.997363
2024-01-25,2467.73,2474.01,2452.36,2470.34,357233504,0.0003,1,0.65,8124338284896,2003599594512515,1.001058
2024-01-26,2458.05,2502.61,2453.45,2478.56,292662412,0.0033,1,8.22,7144493302087,2009937343154269,1.008344
2024-01-29,2490.93,2516.92,2483.81,2500.65,427439639,0.0089,1,22.09,8958520572993,2034462573480314,1.003902
2024-01-30,2523.68,2528.54,2498.81,2498.81,481808939,-0.0007,2,-1.84,8734431084623,2032788176215422,0.990145
2024-01-31,2487.90,2502.45,2486.97,2497.09,454574443,-0.0007,2,-1.72,9000429911787,2030032668710527,1.003694


In [243]:
ref['agr'].iplot(kind='scatter')

In [244]:
data = stock.get_market_ohlcv("20240119", "20240320", ticker, "d")
data['수익률'] = data['종가'] / data['시가']
data = data['수익률']
data

날짜
2024-01-19    1.002119
2024-01-22    1.052854
2024-01-23    1.002008
2024-01-24    0.974052
2024-01-25    0.987705
2024-01-26    1.018868
2024-01-29    1.000000
2024-01-30    0.973251
2024-01-31    0.982906
2024-02-01    1.028384
2024-02-02    1.012739
2024-02-05    0.995807
2024-02-06    1.008457
2024-02-07    1.006289
2024-02-08    0.993902
2024-02-13    0.995902
2024-02-14    1.004237
2024-02-15    1.004228
2024-02-16    1.023158
2024-02-19    1.012295
2024-02-20    0.993927
2024-02-21    0.979633
2024-02-22    1.010571
2024-02-23    1.025157
2024-02-26    0.981595
2024-02-27    0.995833
2024-02-28    1.010571
2024-02-29    0.993711
2024-03-04    0.989451
2024-03-05    0.960334
2024-03-06    0.984615
2024-03-07    0.991071
2024-03-08    1.020270
2024-03-11    0.975610
2024-03-12    0.986364
2024-03-13    1.002304
2024-03-14    0.986207
2024-03-15    1.034965
2024-03-18    0.954751
2024-03-19    1.000000
2024-03-20    1.007075
Name: 수익률, dtype: float64

In [245]:
import pandas as pd

tickers = screening.index
result_df = pd.DataFrame(columns=['날짜'] + list(tickers))

for ticker in tickers:
    data = stock.get_market_ohlcv("20240119", "20240320", ticker, "d")
    data['수익률'] = data['종가'] / data['시가']
    result_df[ticker] = data['수익률']
    if '날짜' not in result_df.columns:
        result_df['날짜'] = data.index

result_df.set_index('날짜', inplace=True)

In [250]:
import pandas as pd

tickers = screening.index
date_range = pd.date_range(start="20240119", end="20240320")
result_df = pd.DataFrame(index=date_range)

for ticker in tickers:
    data = stock.get_market_ohlcv("20240119", "20240320", ticker, "d")
    data['수익률'] = (data['종가'] / data['시가'])
    result_df[ticker] = data['수익률']

# NaN 값을 이전 행의 값으로 채우기
result_df.fillna(method='ffill', inplace=True)

result_df.index.name = '날짜'

In [252]:
result_df['평균수익률'] = result_df.mean(axis=1)
result_df

,011200,000540,044450,053700,091590,004870,290120,050120,073540,평균수익률
날짜,,,,,,,,,,
2024-01-19,0.985136,1.005226,1.017680,1.009091,0.995114,0.985075,1.016644,1.003565,1.002119,1.002183
2024-01-20,0.985136,1.005226,1.017680,1.009091,0.995114,0.985075,1.016644,1.003565,1.002119,1.002183
2024-01-21,0.985136,1.005226,1.017680,1.009091,0.995114,0.985075,1.016644,1.003565,1.002119,1.002183
2024-01-22,0.986323,0.994819,1.005525,0.989324,0.983713,1.011538,0.953457,0.998224,1.052854,0.997309
2024-01-23,1.041009,0.996540,1.010905,1.014414,0.996672,0.947467,1.006974,1.000000,1.002008,1.001777
...,...,...,...,...,...,...,...,...,...,...
2024-03-16,0.951132,0.964546,0.971530,0.985102,0.989267,0.980645,0.993884,1.000000,1.034965,0.985675
2024-03-17,0.951132,0.964546,0.971530,0.985102,0.989267,0.980645,0.993884,1.000000,1.034965,0.985675
2024-03-18,0.996276,0.986842,0.998779,0.992439,0.971377,1.010893,1.013867,1.005725,0.954751,0.992328


In [253]:
result_df['평균수익률'].iplot(kind='scatter')

In [254]:
import plotly.graph_objs as go
import plotly.io as pio

pio.templates["seaborn"] = go.layout.Template(layout=go.Layout(template="seaborn"))
pio.templates.default = "seaborn"

scatter_plot = go.Scatter(x=ref.index, y=ref['agr'], mode='lines', name='코스피 수익률')
scatter_plot_mean = go.Scatter(x=result_df.index, y=result_df['평균수익률'], mode='lines', name='포트폴리오 수익률')

layout = go.Layout(title='<b>3개월 수익률<b>',
                   xaxis=dict(title='날짜'),
                   yaxis=dict(title='수익률'))

# 그래프 객체를 figure로 생성합니다.
fig = go.Figure(data=[scatter_plot, scatter_plot_mean], layout=layout)

# 그래프 출력
fig.show()